## 将“长格式”旋转为“宽格式”

多个时间序列数据通常是以所谓的“长格式”（long）或“堆叠格式”（stacked）存储在数据库和CSV中的。我们先加载一些示例数据，做一些时间序列规整和数据清洗：

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('./macrodata.csv')

data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [4]:
periods = pd.PeriodIndex(year=data.year, 
                         quarter=data.quarter,
                         name='date')

periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [5]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [6]:
data = data.reindex(columns=columns)

data

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [7]:
data.index = periods.to_timestamp('D', 'end')

In [8]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})

ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


这就是多个时间序列（或者其它带有两个或多个键的可观察数据，这里，我们的键是date和item）的长格式。表中的每行代表一次观察。

关系型数据库（如MySQL）中的数据经常都是这样存储的，因为固定架构（即列名和数据类型）有一个好处：随着表中数据的添加，item列中的值的种类能够增加。在前面的例子中，date和item通常就是主键（用关系型数据库的说法），不仅提供了关系完整性，而且提供了更为简单的查询支持。有的情况下，使用这样的数据会很麻烦，你可能会更喜欢DataFrame，不同的item值分别形成一列，date列中的时间戳则用作索引。DataFrame的pivot方法完全可以实现这个转换：

In [10]:
pivoted = ldata.pivot('date', 'item', 'value')

pivoted

/tmp/ipykernel_28403/980140018.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  pivoted = ldata.pivot('date', 'item', 'value')


item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


前两个传递的值分别用作行和列索引，最后一个可选值则是用于填充DataFrame的数据列。假设有两个需要同时重塑的数据列：

In [11]:
ldata['value2'] = np.random.randn(len(ldata))

In [13]:
ldata[0:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.162209
1,1959-03-31 23:59:59.999999999,infl,0.000,0.741288
2,1959-03-31 23:59:59.999999999,unemp,5.800,2.661334
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.480259
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.455482
5,1959-06-30 23:59:59.999999999,unemp,5.100,1.263305
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-0.121759
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.350221
8,1959-09-30 23:59:59.999999999,unemp,5.300,-1.014529
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.718497


如果忽略最后一个参数，得到的DataFrame就会带有层次化的列：

In [15]:
pivoted = ldata.pivot('date', 'item')

pivoted[0:5]

/tmp/ipykernel_28403/240992147.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  pivoted = ldata.pivot('date', 'item')


value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.741288 -0.162209   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.455482  0.480259   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.350221 -0.121759   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -1.077573  0.718497   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.274871 -0.607489   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  2.661334  
1959-06-30 23:59:59.999999999  1.263305  
1959-09-30 23:59:59.999999999 -1.014529  
1959-12-31 23:59:59.999999999 -0.611744  
1960-03-31 23:59:59.999999999  0.961157

In [16]:
pivoted['value'][0:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


注意，pivot其实就是用set_index创建层次化索引，再用unstack重塑：

In [19]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')


unstacked[0:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.741288 -0.162209   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.455482  0.480259   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.350221 -0.121759   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -1.077573  0.718497   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.274871 -0.607489   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2  2.207521 -0.909185   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  0.813646  1.825912   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  2.661334  
1959-06-30 23:59:59.999999999  1.263305  
1959-09-30 23:59:59.999999999 -1.014529  
1959-12-31 23:59:59.999999999 -0.611744  
1960-03-31 23:59:59.999999999  0.961157  
1960-06-30 23:59:59.999999999  0.263422  
1960-09-30 23:59:59.999999999  0.112203

## 将“宽格式”旋转为“长格式”

旋转DataFrame的逆运算是pandas.melt。它不是将一列转换到多个新的DataFrame，而是合并多个列成为一个，产生一个比输入长的DataFrame。看一个例子：

In [21]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]}
                 )

df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


key列可能是分组指标，其它的列是数据值。当使用pandas.melt，我们必须指明哪些列是分组指标。下面使用key作为唯一的分组指标：

In [23]:
melted = pd.melt(df, ['key'])

melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


使用pivot，可以重塑回原来的样子：

In [24]:
reshaped = melted.pivot('key', 'variable', 'value')

reshaped

/tmp/ipykernel_28403/1160189332.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  reshaped = melted.pivot('key', 'variable', 'value')


variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


因为pivot的结果从列创建了一个索引，用作行标签，我们可以使用reset_index将数据移回列：

In [25]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


你还可以指定列的子集，作为值的列：

In [26]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


pandas.melt也可以不用分组指标：

In [27]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [28]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
